In [52]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 100

import yaml

## Localidades

In [191]:
sead_loc = pd.read_csv('../data/seade_codigo_ibge.csv')
br_mun   = pd.read_csv('../data/br_municipios_ibge.csv')
cols = ['geocodigo','nome_mesorregiao','nome_microrregiao']
br_mun = br_mun[cols]

In [196]:
url = "http://api-imp.seade.gov.br/v1/localidade"
r = requests.get(url).json()
df_loc = pd.DataFrame.from_records(r['localidades'])
df_loc = df_loc.merge(sead_loc, how = 'left', left_on='nome', right_on='localidade')
df_loc = df_loc.merge(br_mun, on='geocodigo', how='left')
nomes = ['Total do Estado de São Paulo','Região Metropolitana do Vale do Paraíba e Litoral Norte']
mask = ((df_loc['nome_mesorregiao']=='Vale do Paraíba Paulista') | (df_loc['nome'].isin(nomes)))
# mask = (df_loc['nome'].isin(nomes))

df_loc = df_loc[mask]

In [199]:
df_loc

,codigo,nome,nivel,codigo_pai,nome_pai,geocodigo,localidade,nome_mesorregiao,nome_microrregiao
99,1000,Total do Estado de São Paulo,Estado de São Paulo,0,None,NaN,NaN,NaN,NaN
127,25,Aparecida,Municípios,718,Região de Governo de Guaratinguetá,3502507.0,Aparecida,Vale do Paraíba Paulista,Guaratinguetá
135,576,Arapeí,Municípios,714,Região de Governo de Cruzeiro,3503158.0,Arapeí,Vale do Paraíba Paulista,Bananal
140,35,Areias,Municípios,714,Região de Governo de Cruzeiro,3503505.0,Areias,Vale do Paraíba Paulista,Bananal
155,49,Bananal,Municípios,714,Região de Governo de Cruzeiro,3504909.0,Bananal,Vale do Paraíba Paulista,Bananal
196,85,Caçapava,Municípios,738,Região de Governo de São José dos Campos,3508504.0,Caçapava,Vale do Paraíba Paulista,São José dos Campos
197,86,Cachoeira Paulista,Municípios,718,Região de Governo de Guaratinguetá,3508603.0,Cachoeira Paulista,Vale do Paraíba Paulista,Guaratinguetá
210,97,Campos do Jordão,Municípios,740,Região de Governo de Taubaté,3509700.0,Campos do Jordão,Vale do Paraíba Paulista,Campos do Jordão
213,628,Canas,Municípios,718,Região de Governo de Guaratinguetá,3509957.0,Canas,Vale do Paraíba Paulista,Guaratinguetá
220,105,Caraguatatuba,Municípios,712,Região de Governo de Caraguatatuba,3510500.0,Caraguatatuba,Vale do Paraíba Paulista,Caraguatatuba


## Variaveis

In [202]:
# url =  "http://api-imp.seade.gov.br/v1/variavel?offset=0&limit=100000"
# r = requests.get(url).json()
# df_var = pd.DataFrame.from_records(r['variavel'])

# var_list = list(df_var.keys())

# Get Data

In [203]:
def get_all_cities(locs,anos):
    ### Generate a Clear Dataframe
    df = pd.DataFrame([1])
    df['codigo_localidade']   = 'missing'
    df['localidade']   = 'missing'
    df['nome_localidade_pai']     = 'missing'
    df['codigo_variavel']          = 'missing'
    df['variavel'] = 'missing'
    df['unidade']             = 'missing'
    df['periodo']             = 'missing'
    df['ano']            = 'missing'
    df['loc'] = 'missing'
    df['year']= 'missing'
    df = df.drop(columns=[0])
    df.to_csv("dados_consolidados.csv", index=False, encoding='utf-8')
    
    count = 0
    for localidade in locs:
        print(count,localidade)
        count+=1
        for ano in anos:
            url = "http://api-imp.seade.gov.br/v1/dados/{}/0/{}".format(localidade,ano)
    #           r = requests.get(url).json()
    #             url = "http://api-imp.seade.gov.br/v1/dados/0/2/2010"
            r = requests.get(url).json()
            try:
                df = pd.DataFrame.from_records(r['dados'])
                df['loc'] = localidade
                df['year']= ano

            except:
                df = pd.DataFrame([1])
                df['codigo_localidade']   = 'missing'
                df['localidade']   = 'missing'
                df['nome_localidade_pai']     = 'missing'
                df['codigo_variavel']          = 'missing'
                df['variavel'] = 'missing'
                df['unidade']             = 'missing'
                df['periodo']             = 'missing'
                df['ano']            = 'missing'
                df['loc'] = localidade
                df['year']= ano
                df = df.drop(columns=[0])

    #         df_final = pd.concat([df_final,df], axis=0)
    #         df.to_csv("dados_consolidados_3.csv", index=False, encoding='utf-8')

            df.to_csv("dados_consolidados.csv", index=False, encoding='utf-8', mode='a', header=False)

In [204]:
# var_list
locs = list(df_loc['codigo'].unique())
anos = [i for i in range(2010,2021)]

In [206]:
get_all_cities(locs,anos)

0 1000
1 25
2 576
3 35
4 49
5 85
6 86
7 97
8 628
9 105
10 134
11 136
12 184
13 202
14 204
15 244
16 249
17 263
18 266
19 272
20 317
21 323
22 356
23 380
24 385
25 611
26 419
27 423
28 443
29 460
30 481
31 486
32 495
33 499
34 500
35 507
36 520
37 541
38 548
39 554
40 697


## Manipulate Data

In [207]:
def split_data(df):
    df['ano']  = df['ano'].apply(lambda h: yaml.load(h))
    df['valor'] = df['ano'].apply(lambda x: list(x.values())[0])
    df['ano']  = df['ano'].apply(lambda x: list(x.keys())[0])
    df['valor'] = df['valor'].str.replace('.','').str.replace(',','.')
    
    return(df)

In [208]:
df = pd.read_csv('dados_consolidados.csv')
mask = df['variavel']!='missing'
df = df[mask]

dd = split_data(df)

/home/m/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0,3,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/m/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [217]:
pd.Series(df['variavel'].unique()).sort_values().to_csv('variaveis.csv', index=False)

In [209]:
dd

,codigo_localidade,localidade,nome_localidade_pai,codigo_variavel,variavel,unidade,periodo,ano,loc,year,valor
1,1000,Total do Estado de São Paulo,Não possui nome_localidade_pai,1,Crédito Rural [Série interrompida],(Em reais de 2019),1985/1989-2011,2010,1000,2010,21998277447
2,1000,Total do Estado de São Paulo,Não possui nome_localidade_pai,2,Crédito Rural Agricultura [Série interrompida],(Em reais de 2019),1985/1989-2011,2010,1000,2010,16907984230
3,1000,Total do Estado de São Paulo,Não possui nome_localidade_pai,3,Crédito Rural Pecuária [Série interrompida],(Em reais de 2019),1985/1989-2011,2010,1000,2010,5090293217
4,1000,Total do Estado de São Paulo,Não possui nome_localidade_pai,5,Área,(Em km2),1980-2020,2010,1000,2010,248223.21
5,1000,Total do Estado de São Paulo,Não possui nome_localidade_pai,14,Taxa de Nupcialidade da População de 15 Anos ou Mais,(Por mil habitantes),1980-2018,2010,1000,2010,7.79
...,...,...,...,...,...,...,...,...,...,...,...
267235,697,Região Metropolitana do Vale do Paraíba e Litoral Norte,Total do Estado de São Paulo,1023,População com 60 Anos e Mais,(Em %),1980-2020,2020,697,2020,15.04
267236,697,Região Metropolitana do Vale do Paraíba e Litoral Norte,Total do Estado de São Paulo,1294,População de 6 Anos,Não possui unidade,2000-2020,2020,697,2020,33891
267237,697,Região Metropolitana do Vale do Paraíba e Litoral Norte,Total do Estado de São Paulo,1860,Taxa Geométrica de Crescimento Anual da População - 2010/2020,(Em % a.a.),2020,2020,697,2020,0.96
267238,697,Região Metropolitana do Vale do Paraíba e Litoral Norte,Total do Estado de São Paulo,1922,Taxa Geométrica de Crescimento Anual da População Urbana - 2010/2020,(Em % a.a.),2020,2020,697,2020,1.03


In [210]:
mask = dd['year'].astype(int)>2015
dd = dd[mask]

variaveis = ['Área','População','População de 75 Anos e Mais','PIB', 'PIB per Capita','Empregos Formais', 'Empregos Formais da Indústria',
       'Empregos Formais do Comércio Atacadista e Varejista e do Comércio e Reparação de Veículos Automotores e Motocicletas',
       'Empregos Formais dos Serviços','Matrículas na Pré-escola','Matrículas no Ensino Fundamental','Matrículas no Ensino Médio', 'Médicos Registrados no CRM/SP',
       'Enfermeiros Registrados no COREN/SP','Técnicos de Enfermagem Registrados no COREN/SP','Auxiliares de Enfermagem Registrados no COREN/SP',
       ]

mask = dd['variavel'].isin(variaveis)

dd = dd[mask]

dd = dd.sort_values(by=['codigo_localidade','periodo','nome_localidade_pai','year'], ascending=False).drop_duplicates(['localidade','variavel'],keep='first').sort_values(by='variavel')

In [211]:
dd

,codigo_localidade,localidade,nome_localidade_pai,codigo_variavel,variavel,unidade,periodo,ano,loc,year,valor
91557,202,Igaratá,Região de Governo de São José dos Campos,826,Auxiliares de Enfermagem Registrados no COREN/SP,Não possui unidade,2000-2019,2019,202,2019,39
124185,266,Lavrinhas,Região de Governo de Cruzeiro,826,Auxiliares de Enfermagem Registrados no COREN/SP,Não possui unidade,2000-2019,2019,266,2019,8
6556,1000,Total do Estado de São Paulo,Não possui nome_localidade_pai,826,Auxiliares de Enfermagem Registrados no COREN/SP,Não possui unidade,2000-2019,2019,1000,2019,200228
195967,460,Santa Branca,Região de Governo de São José dos Campos,826,Auxiliares de Enfermagem Registrados no COREN/SP,Não possui unidade,2000-2019,2019,460,2019,47
267137,697,Região Metropolitana do Vale do Paraíba e Litoral Norte,Total do Estado de São Paulo,826,Auxiliares de Enfermagem Registrados no COREN/SP,Não possui unidade,2000-2019,2019,697,2019,8607
58747,628,Canas,Região de Governo de Guaratinguetá,826,Auxiliares de Enfermagem Registrados no COREN/SP,Não possui unidade,2000-2019,2019,628,2019,5
202481,481,Santo Antônio do Pinhal,Região de Governo de Taubaté,826,Auxiliares de Enfermagem Registrados no COREN/SP,Não possui unidade,2000-2019,2019,481,2019,32
143792,323,Natividade da Serra,Região de Governo de Taubaté,826,Auxiliares de Enfermagem Registrados no COREN/SP,Não possui unidade,2000-2019,2019,323,2019,18
19598,576,Arapeí,Região de Governo de Cruzeiro,826,Auxiliares de Enfermagem Registrados no COREN/SP,Não possui unidade,2000-2019,2019,576,2019,6
26096,35,Areias,Região de Governo de Cruzeiro,826,Auxiliares de Enfermagem Registrados no COREN/SP,Não possui unidade,2000-2019,2019,35,2019,7
